In [67]:
import numpy as np

In [68]:
from scipy.stats import mode

In [69]:
GLOBAL_LABEL2ID = {'O': 0,
                   'B-CORP': 1,
                   'B-CW': 2,
                   'B-GRP': 3,
                   'B-LOC': 4,
                   'B-PER': 5,
                   'B-PROD': 6,
                   'I-CORP': 7,
                   'I-CW': 8,
                   'I-GRP': 9,
                   'I-LOC': 10,
                   'I-PER': 11,
                   'I-PROD': 12}
GLOBAL_ID2LABEL = {idx: label for label, idx in GLOBAL_LABEL2ID.items()}

In [76]:
def hard_voting(predictions):
    n = len(predictions)
    labels = []
    for i in range(len(predictions[0])):
        sent_labels = []
        for k in range(len(predictions[0][i])):
            word_labels = []
            for j in range(n):
                word_labels.append(np.argmax(predictions[j][i][k], axis=-1))
            # one model disagrees
            if word_labels[0] == word_labels[1] and word_labels[1] != word_labels[2]:
                if word_labels[0] != 0:
                    label = GLOBAL_ID2LABEL[word_labels[0]]
                elif word_labels[2] != 0:
                    label = GLOBAL_ID2LABEL[word_labels[2]]
                else:
                    label = 'O'
            elif word_labels[1] == word_labels[2] and word_labels[1] != word_labels[0]:
                if word_labels[1] != 0:
                    label = GLOBAL_ID2LABEL[word_labels[1]]
                elif word_labels[0] != 0:
                    label = GLOBAL_ID2LABEL[word_labels[0]]
                else:
                    label = 'O'
            elif word_labels[0] == word_labels[2] and word_labels[1] != word_labels[2]:
                if word_labels[0] != 0:
                    label = GLOBAL_ID2LABEL[word_labels[0]]
                elif word_labels[1] != 0:
                    label = GLOBAL_ID2LABEL[word_labels[1]]
                else:
                    label = 'O'
            # all three disagree
            else:
                label = GLOBAL_ID2LABEL[word_labels[1]]
            sent_labels.append(label)
        labels.append(sent_labels)
    return labels

In [71]:
def soft_voting(predictions):
    n = len(predictions)
    labels = []
    for i in range(len(predictions[0])):
        sent_labels = []
        for k in range(len(predictions[0][i])):
            s = predictions[0][i][k]
            for j in range(1, n):
                for l in range(13):
                    s[l] += predictions[j][i][k][l]
            label = GLOBAL_ID2LABEL[np.argmax(s)]
            sent_labels.append(label)
        labels.append(sent_labels)
    return labels

In [72]:
def save_to_file(filename, preds):
    with open(filename, 'w+') as f:
        for pred in preds:
            f.write('# id\n')
            for label in pred:
                f.write(label + '\n')
            f.write('\n')

In [73]:
import pickle

In [80]:
with open("prediction_rembert/model3_preds", "rb") as fp:
    a = pickle.load(fp)
with open("prediction_rembert/model4_preds", "rb") as fp:
    b = pickle.load(fp)
with open("prediction_rembert/model5_preds", "rb") as fp:
    c = pickle.load(fp)

In [ ]:
def hard_voting(predictions):
    n = len(predictions)
    labels = []
    for i in range(len(predictions[0])):
        sent_labels = []
        for k in range(len(predictions[0][i])):
            word_labels = []
            for j in range(n):
                word_labels.append(np.argmax(predictions[j][i][k], axis=-1))
            label = GLOBAL_ID2LABEL[mode(word_labels).mode.tolist()[0]]
            sent_labels.append(label)
        labels.append(sent_labels)
    return labels

In [62]:
len(a)

8800

In [63]:
len(b)

8800

In [81]:
predictions = [a, b, c]

In [65]:
labels = soft_voting(predictions)
save_to_file('soft.conll', labels)

In [82]:
labels = hard_voting(predictions)
save_to_file('hard_rembert.conll', labels)